# 🚧Tansformaciones en SQL 🖥️

### 🔧Configuración del entorno

In [ ]:
! pip install sqlalchemy pyodbc pandas

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import pyodbc

# 🚀 Parámetros de conexión
server_name = '186.129.9.177,1433'
database_name = 'COMERLOGISTICS'
username = 'Hernan'
password = 'Dapt01'

# 🌐 Crear la conexión con la base de datos
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server')

# 🔄 Cambiar el nombre de la tabla
rename_table_sql = text("""
    EXEC sp_rename 'InvoicePurchases12312016', 'Tabla_Compras';
""")

# 🛠️ Renombrar las columnas según el Siguiente diccionario
rename_columns_sql = text("""
    EXEC sp_rename 'Tabla_Compras.VendorNumber', 'ProveedorID', 'COLUMN';
    EXEC sp_rename 'Tabla_Compras.VendorName', 'Nombre_proveedor', 'COLUMN';
    EXEC sp_rename 'Tabla_Compras.InvoiceDate', 'Fecha_compra', 'COLUMN';
    EXEC sp_rename 'Tabla_Compras.PONumber', 'CompraID', 'COLUMN';
    EXEC sp_rename 'Tabla_Compras.PODate', 'Fecha_ordencompra', 'COLUMN';
    EXEC sp_rename 'Tabla_Compras.PayDate', 'Fecha_pago', 'COLUMN';
    EXEC sp_rename 'Tabla_Compras.Quantity', 'Cantidad', 'COLUMN';
    EXEC sp_rename 'Tabla_Compras.Dollars', 'Importe', 'COLUMN';
    EXEC sp_rename 'Tabla_Compras.Freight', 'Costo_envio', 'COLUMN';
    EXEC sp_rename 'Tabla_Compras.Approval', 'Aprobacion', 'COLUMN';
""")

# 🔄 Establece la columna como NOT NULL:
table_columns_notnull = text("""
    ALTER TABLE Tabla_Compras
ALTER COLUMN CompraID INT NOT NULL;
""")

# 🗝️ Establecer la columna 'CompraID' como clave primaria
set_primary_key_sql = text("""
    ALTER TABLE Tabla_Compras
    ADD CONSTRAINT PK_CompraID PRIMARY KEY (CompraID);
""")

# 💾 Ejecutar los comandos SQL
with engine.connect() as connection:
    print("➡️ Ejecutando cambio de nombre de la tabla...")
    connection.execute(rename_table_sql)
    print("✅ Cambio de nombre de la tabla ejecutado.")
    connection.commit()

    print("➡️ Ejecutando cambio de nombre de las columnas...")
    connection.execute(rename_columns_sql)
    print("✅ Cambio de nombre de las columnas ejecutado.")
    connection.commit()

    print("➡️ Estableciendo columna CompraID como NOT NULL...")
    connection.execute(table_columns_notnull)
    print("✅ Columna CompraID establecida como NOT NULL.")
    connection.commit()

    print("➡️ Estableciendo clave primaria en la columna CompraID...")
    connection.execute(set_primary_key_sql)
    print("✅ Clave primaria establecida.")
    connection.commit()

# 🔒 Cerrar la conexión si ya no se va a utilizar
engine.dispose()
print("🔒 Conexión cerrada.")

-----

In [ ]:
from sqlalchemy import create_engine, text

# 🚀 Parámetros de conexión
server_name = '186.129.9.177,1433'
database_name = 'COMERLOGISTICS'
username = 'Marcelo'
password = 'Dapt01'

# 🌐 Crear la conexión con la base de datos
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server')

# 💾 Ejecutar los comandos SQL con commits explícitos
with engine.connect() as connection:
    try:
        print("➡️ Ejecutando cambio de nombre de la tabla...")
        rename_table_sql = text("EXEC sp_rename 'SalesFINAL12312016', 'Tabla_VentasFinal';")
        connection.execute(rename_table_sql)
        connection.commit()
        print("✅ Cambio de nombre de la tabla ejecutado.")

        print("➡️ Ejecutando cambio de nombre de las columnas...")
        rename_columns_sql = [
            "EXEC sp_rename 'Tabla_VentasFinal.InventoryId', 'InventarioID', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.Store', 'Tienda', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.Brand', 'MarcaID', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.Description', 'Descripcion', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.Size', 'Tamaño', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.SalesQuantity', 'Cantidad_Ventas', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.SalesDollars', 'Venta_Total', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.SalesPrice', 'Precio_Unitario', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.SalesDate', 'Fecha_Venta', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.Volume', 'Volumen', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.Classification', 'Clasificacion', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.ExciseTax', 'Impuesto', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.VendorNo', 'ProveedorID', 'COLUMN'",
            "EXEC sp_rename 'Tabla_VentasFinal.VendorName', 'Nombre_Proveedor', 'COLUMN'"
        ]
        for sql in rename_columns_sql:
            connection.execute(text(sql))
        connection.commit()
        print("✅ Cambio de nombre de las columnas ejecutado.")

        print("➡️ Creando nuevas columnas...")
        add_columns_sql = text("""
            ALTER TABLE Tabla_VentasFinal
            ADD Inventario_inicialID INT,
                VentasID INT IDENTITY(1,1);
        """)
        connection.execute(add_columns_sql)
        connection.commit()
        print("✅ Nuevas columnas creadas.")

        print("➡️ Asignando valores únicos a Inventario_inicialID...")
        assign_values_sql = text("""
            ;WITH CTE AS (
                SELECT ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS row_num, InventarioID
                FROM Tabla_VentasFinal
            )
            UPDATE Tabla_VentasFinal
            SET Inventario_inicialID = CTE.row_num
            FROM Tabla_VentasFinal
            JOIN CTE ON Tabla_VentasFinal.InventarioID = CTE.InventarioID;
        """)
        connection.execute(assign_values_sql)
        connection.commit()
        print("✅ Valores únicos asignados.")

    except Exception as e:
        print(f"Error: {e}")
        connection.rollback()  # Deshacer la transacción si hay un error

# 🔒 Cerrar la conexión
engine.dispose()
print("🔒 Conexión cerrada.")



In [15]:
import pandas as pd
from sqlalchemy import create_engine, text
import pyodbc

# 🚀 Parámetros de conexión
server_name = '186.129.9.177,1433'
database_name = 'COMERLOGISTICS'
username = 'Ruben'
password = 'Dapt01'

# 🌐 Crear la conexión con la base de datos
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server')

# 🔄 Cambiar el nombre de la tabla (si es necesario)
rename_table_sql = text("EXEC sp_rename 'PurchasesFINAL12312016', 'Tabla_Detallecompras';")

# 🛠️ Comandos para renombrar las columnas según el diccionario proporcionado
rename_columns_sql_commands = [
    "EXEC sp_rename 'Tabla_Detallecompras.InventoryId', 'Inventario_inicialID', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.Store', 'Tienda', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.Brand', 'MarcaID', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.Description', 'Descripcion', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.Size', 'Tamaño', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.VendorNumber', 'ProveedorID', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.VendorName', 'Nombre_Proveedor', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.PONumber', 'CompraID', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.PODate', 'Fecha_ordencompra', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.ReceivingDate', 'Fecha_recepcion', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.InvoiceDate', 'Fecha_compra', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.PayDate', 'Fecha_pago', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.PurchasePrice', 'Precio_compra', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.Quantity', 'Cantidad', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.Dollars', 'Importe', 'COLUMN';",
    "EXEC sp_rename 'Tabla_Detallecompras.Classification', 'Clasificacion', 'COLUMN';"
]

# 🔄 Crear la columna 'Detalle_compraID' y llenarla con valores incrementales
add_detalle_compraID_sql = text("""
    ALTER TABLE Tabla_Detallecompras ADD Detalle_compraID INT IDENTITY(1,1);
""")

# 🗝️ Establecer la columna 'Detalle_compraID' como clave primaria
set_primary_key_sql = text("""
    ALTER TABLE Tabla_Detallecompras
    ADD CONSTRAINT PK_Detalle_compraID PRIMARY KEY (Detalle_compraID);
""")

# 🔗 Establecer las claves foráneas en 'Inventario_inicialID', 'MarcaID', 'CompraID'
set_foreign_keys_sql_commands = [
    "ALTER TABLE Tabla_Detallecompras ADD CONSTRAINT FK_Inventario_inicialID FOREIGN KEY (Inventario_inicialID) REFERENCES Tabla_InventarioInicial(Inventario_inicialID);",
    "ALTER TABLE Tabla_Detallecompras ADD CONSTRAINT FK_MarcaID FOREIGN KEY (MarcaID) REFERENCES Productos(MarcaID);",
    "ALTER TABLE Tabla_Detallecompras ADD CONSTRAINT FK_CompraID FOREIGN KEY (CompraID) REFERENCES Tabla_Compras(CompraID);"
]

# 💾 Ejecutar los comandos SQL
with engine.connect() as connection:
    print("➡️ Ejecutando cambio de nombre de la tabla...")
    connection.execute(rename_table_sql)
    print("✅ Cambio de nombre de la tabla ejecutado.")
    connection.commit()

    print("➡️ Ejecutando cambio de nombre de las columnas...")
    for sql in rename_columns_sql_commands:
        connection.execute(text(sql))
    print("✅ Cambio de nombre de las columnas ejecutado.")
    connection.commit()

    print("➡️ Creando la columna Detalle_compraID con valores incrementales...")
    connection.execute(add_detalle_compraID_sql)
    print("✅ Columna Detalle_compraID creada con valores incrementales.")
    connection.commit()

    print("➡️ Estableciendo clave primaria en la columna Detalle_compraID...")
    connection.execute(set_primary_key_sql)
    print("✅ Clave primaria establecida.")
    connection.commit()

    print("➡️ Estableciendo claves foráneas...")
    for sql in set_foreign_keys_sql_commands:
        connection.execute(text(sql))
    print("✅ Claves foráneas establecidas.")
    connection.commit()

# 🔒 Cerrar la conexión si ya no se va a utilizar
engine.dispose()
print("🔒 Conexión cerrada.")


➡️ Ejecutando cambio de nombre de la tabla...


ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]No item by the name of 'PurchasesFINAL12312016' could be found in the current database 'COMERLOGISTICS', given that @itemtype was input as '(null)'. (15225) (SQLExecDirectW)")
[SQL: EXEC sp_rename 'PurchasesFINAL12312016', 'Tabla_Detallecompras';]
(Background on this error at: https://sqlalche.me/e/20/f405)

-----

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import pyodbc

# 🚀 Parámetros de conexión
server_name = '186.129.9.177,1433'
database_name = 'COMERLOGISTICS'
username = 'Carlos'
password = 'Dapt01'

# 🌐 Crear la conexión con la base de datos
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server')


# 🔄 Establece la columna como NOT NULL:
table_columns_notnull = text("""
    ALTER TABLE Tabla_InventarioInicial
ALTER COLUMN Inventario_inicialID  VARCHAR(255) NOT NULL;
""")

# 🗝️ Establecer la columna 'Invetario_InicialID' como clave primaria
set_primary_key_sql = text("""
    ALTER TABLE Tabla_InventarioInicial
 ADD CONSTRAINT PK_Inventario_inicialID PRIMARY KEY (Inventario_inicialID);
""")

# 💾 Ejecutar los comandos SQL
with engine.connect() as connection:
    print("➡️ Estableciendo columna Inventario_InicialID como NOT NULL...")
    connection.execute(table_columns_notnull)
    print("✅ Columna Inventario_InicialID establecida como NOT NULL.")
    connection.commit()

    print("➡️ Estableciendo clave primaria en la columna Inventario_InicialID...")
    connection.execute(set_primary_key_sql)
    print("✅ Clave primaria establecida.")
    connection.commit()

# 🔒 Cerrar la conexión si ya no se va a utilizar
engine.dispose()
print("🔒 Conexión cerrada.")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import pyodbc

# 🚀 Parámetros de conexión
server_name = '186.129.9.177,1433'
database_name = 'COMERLOGISTICS'
username = 'Daniela'
password = 'Dapt01'

# 🌐 Crear la conexión con la base de datos
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server')


# 🔄 Establece la columna como NOT NULL:
table_columns_notnull = text("""
    ALTER TABLE dbo.Productos
    ALTER COLUMN MarcaID INT NOT NULL;
""")

# 🗝️ Establecer la columna 'Invetario_InicialID' como clave primaria
set_primary_key_sql = text("""
    ALTER TABLE dbo.Productos
    ADD CONSTRAINT PK_MarcaID PRIMARY KEY (MarcaID);
""")

# 💾 Ejecutar los comandos SQL
with engine.connect() as connection:
    print("➡️ Estableciendo columna MarcaID como NOT NULL...")
    connection.execute(table_columns_notnull)
    print("✅ Columna MarcaID establecida como NOT NULL.")
    connection.commit()

    print("➡️ Estableciendo clave primaria en la columna MarcaID...")
    connection.execute(set_primary_key_sql)
    print("✅ Clave primaria establecida.")
    connection.commit()

# 🔒 Cerrar la conexión si ya no se va a utilizar
engine.dispose()
print("🔒 Conexión cerrada.")